# Introducción básica a Redes Neuronales

Vamos a usar Scikit-learn para tomar contacto con las redes neuronales. Aunque no se recomienda para hacer Deep Learning, porque no tiene soporte de GPUs, nos va a servir para tomar contacto inicialmente con redes neuronales con un interfaz que ya conocemos de las clases de Supervised Learning.

Primero, cargamos los módulos necesarios:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams["savefig.dpi"] = 300
plt.rcParams["savefig.bbox"] = "tight"
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler

## Clasificación


Cargamos los módulos necesario de redes neuronales de Scikit-Learn y también cargamos el dataset de pruebas Make Moons para hacer una prueba de clasificación y lo pintamos:

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=100, noise=0.25, random_state=2)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    random_state=42)

plt.scatter(X_train[:, 0], X_train[:, 1], c=plt.cm.tab10(y_train))
xlim = plt.xlim()
ylim = plt.ylim()

In [ ]:
xs = np.linspace(xlim[0], xlim[1], 1000)
ys = np.linspace(ylim[0], ylim[1], 1000)
xx, yy = np.meshgrid(xs, ys)
X_grid = np.c_[xx.ravel(), yy.ravel()]

Por defecto, esto está utilizando una no linealidad con ReLU, pero trozo a trozo la frontera de decisión será lineal. Ya que el dataset es muy pequeño hemos puesto como solver l-bfgs aunque por defecto el solver es adam:

In [ ]:
mlp = MLPClassifier(solver='lbfgs', random_state=0).fit(X_train, y_train)
print(mlp.score(X_train, y_train))
print(mlp.score(X_test, y_test))

Pintamos la frontera de decisión

In [ ]:
plt.contour(xx, yy, mlp.predict_proba(X_grid)[:, 1].reshape(xx.shape), levels=[.5])
plt.scatter(X_train[:, 0], X_train[:, 1], c=plt.cm.tab10(y_train))

plt.xlim(xlim)
plt.ylim(ylim)

Como ya hemos dicho, ya que se trata de un problema no convexo el resultado dependerá de la inicialización (pese a que el solver elegido es bastante robusto). Vamos a comprobarlo introducciones variación en el random state utilizado:

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(8, 5))
for ax, i in zip(axes.ravel(), range(10)):
    mlp = MLPClassifier(solver='lbfgs', random_state=i, max_iter=1000).fit(X_train, y_train)
    print(mlp.score(X_train, y_train))
    print(mlp.score(X_test, y_test))

    ax.contour(xx, yy, mlp.predict_proba(X_grid)[:, 1].reshape(xx.shape), levels=[.5])
    ax.scatter(X_train[:, 0], X_train[:, 1], c=plt.cm.tab10(y_train))

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xticks(())
    ax.set_yticks(())

Esta red tiene por defecto una única capa de 100 nodos, con lo que la red es demasiado flexible para un dataset tan pequeño y lo que está haciendo es un sobreajuste para todos los diversos random states que hemos utilizado.

Vamos a reducir tamaño de las capas ocultas (3 en este caso) para reducir el grado de sobreajuste que estamos viendo:

In [ ]:
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10, 10, 10), random_state=0)
mlp.fit(X_train, y_train)
print(mlp.score(X_train, y_train))
print(mlp.score(X_test, y_test))

En este caso, los tamaños de las matrices de pesos son consecutivamente 2x10, 10x10, 10x10 y 10x1:

In [ ]:
plt.contour(xx, yy, mlp.predict_proba(X_grid)[:, 1].reshape(xx.shape), levels=[.5])
plt.scatter(X_train[:, 0], X_train[:, 1], c=plt.cm.tab10(y_train))

plt.xlim(xlim)
plt.ylim(ylim)

En el gráfico pueden apreciarse los 10 segmentos correspondientes a los diez nodos que tienen las capas ocultas de la red neuronal. Como hemos dicho, no suele ser así como se configuran redes neuronales: típicamente las arquitecturas son mucho más complejas como para que la red pueda aprender funciones arbitrariamente complejas y que hacen que a la vez la red neuronal pueda generalizar suficientemente bien.


Si utilizamos una función de activación `tanh` se puede ver que los resultados son mucho más suaves:

También podemos cambiar la función de activación, 

In [ ]:
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10, 10, 10), activation="tanh", random_state=0)
mlp.fit(X_train, y_train)
print(mlp.score(X_train, y_train))
print(mlp.score(X_test, y_test))

In [ ]:
plt.contour(xx, yy, mlp.predict_proba(X_grid)[:, 1].reshape(xx.shape), levels=[.5])
plt.scatter(X_train[:, 0], X_train[:, 1], c=plt.cm.tab10(y_train))

plt.xlim(xlim)
plt.ylim(ylim)

En aplicaciones reales, la suavidad de la frontera de decisión no importa tanto. Importa más tener una *learning rate* más alta, por ejemplo, y los data scientist utilizarán preferentemente una activación ReLU. 

## Regresión

Abordamos ahora un problema de regresión creado un dataset que sigue una función con aleatoriedad en las muestras, para ver qué tal predice la función original la red neuronal:

In [ ]:
rng = np.random.RandomState(0)
x = np.sort(rng.uniform(size=100))
y = np.sin(10 * x) + 5 * x + np.random.normal(0, .3, size=100)
plt.plot(x, y, 'o')

In [ ]:
line = np.linspace(0, 1, 100)
X = x.reshape(-1, 1)

Aquí estamos utilizando un MLPRegressor (Multilayer Perceptron Regressor), que optimiz el error cuadrádrico utilizando o l-bfgs, *stochastic gradient descent* o la opción por defecto, *adam*. En este caso, debido al tamaño del dataset elegimos l-bfgs, y configuramos dos modelos para cada tipo de activación (`relu` o `tanh`):

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp_relu = MLPRegressor(solver="lbfgs", hidden_layer_sizes=(10, 10, 10), max_iter=4000).fit(X, y)
mlp_tanh = MLPRegressor(solver="lbfgs", hidden_layer_sizes=(10, 10, 10), activation='tanh',  max_iter=4000).fit(X, y)

In [ ]:
plt.plot(x, y, 'o')
plt.plot(line, mlp_relu.predict(line.reshape(-1, 1)), label="relu")
plt.plot(line, mlp_tanh.predict(line.reshape(-1, 1)), label="tanh")
plt.legend()

# Control de la complejidad

Primero, ver las slides para la introducción de algunos conceptos.

Para mostrar la influencia de los parámetros sobre la red neuronal vamos a hacer un grid search de Scikit-Learn y utilizar diferentes parámetros.

Seleccionamos para la prueba el Breast Cancer Dataset, con el que también trabajaremos después con Keras:

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

Preparamos los datos, estratificándolos y utilizando un standard scaler:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, stratify=data.target, random_state=0)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Configuramos y entrenamos una Multilayer Perceptron Network utilizando los valores por defecto (una sola capa, con 100 nodos, adam optimizer y activación ReLu):

In [ ]:
mlp = MLPClassifier(max_iter=1000, random_state=0).fit(X_train_scaled, y_train)
print(mlp.score(X_train_scaled, y_train))
print(mlp.score(X_test_scaled, y_test))

In [ ]:
mlp = MLPClassifier(solver="lbfgs", random_state=1).fit(X_train_scaled, y_train)
print(mlp.score(X_train_scaled, y_train))
print(mlp.score(X_test_scaled, y_test))

El parámetro de regularización (*weight decay*) se llama *alpha* como en Ridge. Hacemos un grid search sobre este parámetro utilizando las pipelines de Scikit-learn para ver si encontramos un buen valor:

In [ ]:
from sklearn.model_selection import GridSearchCV
pipe = make_pipeline(StandardScaler(), MLPClassifier(max_iter=2000, solver="lbfgs", random_state=1))
param_grid = {'mlpclassifier__alpha': np.logspace(-3, 3, 7)}
grid = GridSearchCV(pipe, param_grid, return_train_score=True)

In [ ]:
grid.fit(X_train, y_train)

Almacenamos los resultados del grid search en un dataframe de pandas:

In [ ]:
results = pd.DataFrame(grid.cv_results_)
res = results.pivot_table(index="param_mlpclassifier__alpha",
                          values=["mean_test_score", "mean_train_score"])
res

Pintamos el resultado:

In [ ]:
res.plot()
plt.xscale("log")
plt.ylim(0.95, 1.01)

Como era de esperar, la precisión de training baja con el incremento de regularización. Vamos a incluir unas barras de error sobre capa punto para poder discutir mejor el problema:

In [ ]:
res = results.pivot_table(index="param_mlpclassifier__alpha",
                          values=["mean_test_score",
                                  "mean_train_score",
                                  "std_test_score",
                                  "std_train_score"])

In [ ]:
res.mean_test_score.plot(yerr=res.std_test_score)
res.mean_train_score.plot(yerr=res.std_train_score)
plt.xscale("log")
plt.ylim(0.95, 1.01)
plt.legend()

Las barras de error son bastante grandes, pero pese a eso parece que el óptimo está para un valor de *alpha* de 10, con una media más grande aunque con un poco más de desviación estándar que si hubiésemos un valor de *alpha* de $10^{-2}$.

También podemos hacer un grid search sobre el tamaño de las capas ocultas (podríamos buscar ambas a la vez tmabién):

In [ ]:
from sklearn.model_selection import GridSearchCV
pipe = make_pipeline(StandardScaler(), MLPClassifier(solver="lbfgs", random_state=1))
param_grid = {'mlpclassifier__hidden_layer_sizes':
              [(10,), (50,), (100,), (500,), (10, 10), (50, 50), (100, 100), (500, 500)]
             }
grid = GridSearchCV(pipe, param_grid, return_train_score=True)

Nada obliga a elegir diferentes capas con el mismo tamaño en número de nodos, aunque en datos tabulados es lo que se suele hacer porque simplifica algo las cosas.

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
results = pd.DataFrame(grid.cv_results_)
res = results.pivot_table(index="param_mlpclassifier__hidden_layer_sizes",
                          values=["mean_test_score",
                                  "mean_train_score",
                                  "std_test_score",
                                  "std_train_score"])

In [ ]:
res.mean_test_score.plot(yerr=res.std_test_score)
res.mean_train_score.plot(yerr=res.std_train_score)
plt.legend()

Lo que puede verse en este caso es que 100 nodos es lo que parece mejor, y en cualquier caso funciona mejor una configuración con una sola capa oculta en vez de dos.

Esto son pruebas *de juguete* para entender los conceptos. Scikit-Learn no escala demasiado bien; para un minidataset como éste está bien, pero para cosas más serias (por ejemplo, el procesado de MNIST) no lo vamos a usar y pasaremos a otros frameworks que sí son utilizados de manera profesional.

Ahora, volver a las transparencias de la presentación para una recapitulació del apartado de control de complejiad.

# Más allá de Scikit-Learn

Se incluyen aquí la definición de un par de clases que implementan de una manera básica el *forward* pass y el *backward* pass de una red neuronal, así como dos operaciones de grafo computacional, una unión (enrutador de gradiente) y una multiplicación (conmutador ponderado de gradiente):

In [ ]:
class NeuralNetwork(object):
    def __init__(self):
        # initialize coefficients and biases
        pass
    def forward(self, x):
        activation = x
        for coef, bias in zip(self.coef_, self.bias_):
            activation = self.nonlinearity(np.dot(activation, coef) + bias)
        return activation
    def backward(self, x):
        # compute gradient of stuff in forward pass
        pass

In [ ]:
# http://mxnet.io/architecture/program_model.html
class array(object) :
    """Simple Array object that support autodiff."""
    def __init__(self, value, name=None):
        self.value = value
        if name:
            self.grad = lambda g : {name : g}

    def __add__(self, other):
        assert isinstance(other, int)
        ret = array(self.value + other)
        ret.grad = lambda g : self.grad(g)
        return ret

    def __mul__(self, other):
        assert isinstance(other, array)
        ret = array(self.value * other.value)
        def grad(g):
            x = self.grad(g * other.value)
            x.update(other.grad(g * self.value))
            return x
        ret.grad = grad
        return ret

In [ ]:
# some examples
a = array(np.array([1, 2]), 'a')
b = array(np.array([3, 4]), 'b')
c = b * a
d = c + 1
print(d.value)
print(d.grad(1))